In [ ]:
from ImageLoader import *
from NoiseRemoval import *
from Segmentation import *
from OrientationDetector import *
from LineCutter import *
from WordCutter import *
from CharCutter import *
from FeatureExtractor import *
from Classifier import *
from Preprocessing import *
import numpy as np
# plt.rc("figure", dpi=1000)

In [ ]:
path = ".\\fonts-dataset\\Lemonada\\"
image = ImageLoader.loadImage(path,"1.jpeg")
alteredImage=image
ImageLoader.print(alteredImage)
# alteredImage = NoiseRemoval.applyGaussianBlur(image=image)
# ImageLoader.print(alteredImage,sep=False)
# alteredImage = NoiseRemoval.sharpenImage(alteredImage)
alteredImage = Segmentation.segment(alteredImage)
ImageLoader.print(alteredImage)
alteredImage = OrientationDetector.rotate(alteredImage)
ImageLoader.print(alteredImage)
alteredImage = LineCutter.extractLines(alteredImage)
ImageLoader.print(alteredImage)
alteredImage = WordCutter.extractWords(alteredImage[4])
#alteredImage = CharCutter.extractCharacters(alteredImage)
# cv2.imshow("",alteredImage[0])
# cv2.waitKey(0)
#print(alteredImage)
ImageLoader.print(alteredImage)
for i in range(0,len(alteredImage)):
    alteredImage[i] = alteredImage[i].tolist()

In [ ]:
#ImageLoader.print(alteredImage)
alteredImage2 = CharCutter.extractCharacters(alteredImage)
# for ii in alteredImage2:
#     for jj in ii:
        # print(jj.StartIndex,jj.CutIndex,jj.EndIndex)
# print(alteredImage2)
#ImageLoader.print(alteredImage2,sep=True)



# ImageLoader.print(alteredImage2)
# lll = [[[1,2,3],[4,5,6],[7,8,9]],[[10,11,12],[13,14,15],[16,17,18]]]
# print(lll)

In [ ]:
feature_extractor = FeatureExtractor()
feature_extractor.loadDataset(".\\char_sample\\")
features,labels = feature_extractor.extractFeatures(method='HOG')
classifier = Classifier()

print(len(features))
print(len(labels))

classifier.train(features,labels)

In [ ]:
ImageLoader.print(Preprocessing.crop_to_fit_white_with_border(alteredImage2[1]))
result = classifier.classify(Preprocessing.crop_to_fit_white_with_border(alteredImage2[1]),feature_extractor,method='HOG')
print(result)